In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

PROJECT_ROOT = "/content/drive/MyDrive/GalaxEye_ship_detection/galaxeye"
os.chdir(PROJECT_ROOT)

print("Current working directory:", os.getcwd())
print("Files:", os.listdir("."))

Current working directory: /content/drive/MyDrive/GalaxEye_ship_detection/galaxeye
Files: ['scripts', 'requirements.txt', 'yolov8s.pt', 'results', 'runs', 'data']


In [ ]:
!pip install --upgrade pip
!pip install ultralytics opencv-python matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 33.0 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 10.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [ultralytics]


In [ ]:
!yolo version

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
8.4.7


### Quick dataset check

In [ ]:
import yaml

with open("data/ship.yaml") as f:
    data_cfg = yaml.safe_load(f)

data_cfg

{'path': 'data/yolo',
 'train': 'images/train',
 'val': 'images/val',
 'nc': 1,
 'names': ['ship']}

### Start training

In [ ]:
!yolo detect train \
  model=yolov8s.pt \
  data=data/ship.yaml \
  epochs=100 \
  imgsz=640 \
  batch=16 \
  optimizer=AdamW \
  lr0=0.001 \
  patience=20 \
  project=runs \
  name=ssdd_yolov8s \
  device=0

Ultralytics 8.4.7 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data/ship.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=ssdd_yolov8s, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=True, patience=20, perspective=0.0, plots=True, pose=12

### Monitor results files

In [ ]:
!ls runs/detect/runs/ssdd_yolov8s

args.yaml			 results.csv	      val_batch0_labels.jpg
BoxF1_curve.png			 results.png	      val_batch0_pred.jpg
BoxP_curve.png			 train_batch0.jpg     val_batch1_labels.jpg
BoxPR_curve.png			 train_batch1.jpg     val_batch1_pred.jpg
BoxR_curve.png			 train_batch2.jpg     val_batch2_labels.jpg
confusion_matrix_normalized.png  train_batch5220.jpg  val_batch2_pred.jpg
confusion_matrix.png		 train_batch5221.jpg  weights
labels.jpg			 train_batch5222.jpg


In [ ]:
!ls runs/detect/runs/ssdd_yolov8s/weights

best.pt  last.pt


### Validation (Quantitative Evaluation)

In [ ]:
!yolo detect val \
  model=runs/detect/runs/ssdd_yolov8s/weights/best.pt \
  data=data/ship.yaml \
  imgsz=640

Ultralytics 8.4.7 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 73 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs
val: Fast image access ✅ (ping: 0.7±0.4 ms, read: 17.0±12.3 MB/s, size: 33.3 KB)
val: Scanning /content/drive/MyDrive/GalaxEye_ship_detection/galaxeye/data/yolo/labels/val.cache... 232 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 232/232 25.6Mit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 15/15 3.9it/s 3.9s
                   all        232        546      0.967      0.972      0.986      0.748
Speed: 2.0ms preprocess, 8.1ms inference, 0.0ms loss, 1.7ms postprocess per image
Results saved to /content/drive/MyDrive/GalaxEye_ship_detection/galaxeye/runs/detect/val
💡 Learn more at https://docs.ultralytics.com/modes/val


### (Optional but STRONG) Inference on SSDD sample

In [ ]:
!yolo detect predict \
  model=runs/detect/runs/ssdd_yolov8s/weights/best.pt \
  source=data/yolo/images/val \
  imgsz=640 \
  conf=0.25

Ultralytics 8.4.7 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 73 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs

image 1/232 /content/drive/MyDrive/GalaxEye_ship_detection/galaxeye/data/yolo/images/val/000001.jpg: 512x640 1 ship, 45.5ms
image 2/232 /content/drive/MyDrive/GalaxEye_ship_detection/galaxeye/data/yolo/images/val/000009.jpg: 512x640 1 ship, 13.0ms
image 3/232 /content/drive/MyDrive/GalaxEye_ship_detection/galaxeye/data/yolo/images/val/000011.jpg: 544x640 1 ship, 45.3ms
image 4/232 /content/drive/MyDrive/GalaxEye_ship_detection/galaxeye/data/yolo/images/val/000019.jpg: 544x640 1 ship, 15.7ms
image 5/232 /content/drive/MyDrive/GalaxEye_ship_detection/galaxeye/data/yolo/images/val/000021.jpg: 544x640 1 ship, 15.6ms
image 6/232 /content/drive/MyDrive/GalaxEye_ship_detection/galaxeye/data/yolo/images/val/000029.jpg: 512x640 1 ship, 13.5ms
image 7/232 /content/drive/MyDrive/GalaxEye_ship_detection/galaxeye/data/yolo/images/val/000